# Fetch Open Datasets of PV locations

Most of these datasets are located in [Zenodo](https://zenodo.org/), a general-purpose open-access repository developed under the European OpenAIRE program and operated by CERN. The datasets are available in various formats, including CSV, GeoJSON, and shapefiles, and we'll be using open-source Python libraries to download and process them.

Here we list the dataset titles alongside their first author, DOI links, and their number of labels:
- "A solar panel dataset of very high resolution satellite imagery to support the Sustainable Development Goals" - C. Clark et al, 2023 | [paper DOI](https://doi.org/10.1038/s41597-023-02539-8) | [dataset DOI](https://doi.org/10.6084/m9.figshare.22081091.v3) | 2,542 object labels (per spatial resolution)
- "A global inventory of photovoltaic solar energy generating units" - L. Kruitwagen et al, 2021 | [paper DOI](https://doi.org/10.1038/s41586-021-03957-7) | [dataset DOI](https://doi.org/10.5281/zenodo.5005867) | 50,426 for training, cross-validation, and testing; 68,661 predicted polygon labels 
- "A harmonised, high-coverage, open dataset of solar photovoltaic installations in the UK" - D. Stowell et al, 2020 | [paper DOI](https://doi.org/10.1038/s41597-020-00739-0) | [dataset DOI](https://zenodo.org/records/4059881) | 265418 data points (over 255,000 are stand-alone installations, 1067 solar farms, and rest are subcomponents within solar farms)
- "A crowdsourced dataset of aerial images with annotated solar photovoltaic arrays and installation metadata" - G. Kasmi, 2023 | [paper DOI](https://doi.org/10.1038/s41597-023-01951-4) | [dataset DOI](https://doi.org/10.5281/zenodo.6865878) | > 28K points of PV installations; 13K+ segmentation masks for PV arrays; metadata for 8K+ installations
- "Georectified polygon database of ground-mounted large-scale solar photovoltaic sites in the United States" - K. Sydny, 2023 | [paper DOI](https://doi.org/10.1038/s41597-023-02644-8) | [dataset DOI](https://www.sciencebase.gov/catalog/item/6671c479d34e84915adb7536) | 4186 data points (Note: these correspond to PV _facilities_ rather than individual panel arrays or objects and need filtering of duplicates with other datasets and further processing to extract the PV arrays in the facility)
- "Vectorized solar photovoltaic installation dataset across China in 2015 and 2020" - J. Liu et al, 2024 | [paper DOI](https://doi.org/10.1038/s41597-024-04356-z) | [dataset link](https://github.com/qingfengxitu/ChinaPV) | 3,356 PV labels (inspect quality!)
- "Multi-resolution dataset for photovoltaic panel segmentation from satellite and aerial imagery" - H. Jiang, 2021 | [paper DOI](https://doi.org/10.5194/essd-13-5389-2021) | [dataset DOI](https://doi.org/10.5281/zenodo.5171712) | 3716 samples of PV data points
- "An Artificial Intelligence Dataset for Solar Energy Locations in India" - A. Ortiz, 2022 | [paper DOI](https://doi.org/10.1038/s41597-022-01499-9) | [dataset link 1](https://researchlabwuopendata.blob.core.windows.net/solar-farms/solar_farms_india_2021.geojson) or [dataset link 2](https://raw.githubusercontent.com/microsoft/solar-farms-mapping/refs/heads/main/data/solar_farms_india_2021_merged_simplified.geojson) | 117 geo-referenced points of solar installations across India
- "GloSoFarID: Global multispectral dataset for Solar Farm IDentification in satellite imagery" - Z. Yang, 2024 | [paper DOI](https://doi.org/10.48550/arXiv.2404.05180) | [dataset DOI](https://doi.org/10.5281/zenodo.10939099) | 6793 PV samples across 3 years (double counting of samples)
- "Distributed solar photovoltaic array location and extent dataset for remote sensing object identification" - K. Bradbury, 2016 | [paper DOI](https://doi.org/10.1038/sdata.2016.106) | [dataset DOI](https://doi.org/10.6084/m9.figshare.3385780.v4) | polygon annotations for 19,433 PV modules in 4 cities in California, USA

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from dotenv import load_dotenv
from tqdm import tqdm

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import geopandas as gpd
import rasterio
import shapely
import pygeohash
import folium
import lonboard
# import openeo 
# import easystac
# import pydeck
# import easystac
# import cubo

import duckdb as dd 
import datahugger
import sciencebasepy

import os

In [ ]:
# create dict of metadata for datasets
# this will be used for interactive widget and managing downloads

# for maxar dataset
# Catalogue ID 1040050029DC8C00; use to find geospatial extent coords
# The geocoordinates for each solar panel object may be determined using the native resolution labels (found in the labels_native directory). 
# The center and width values for each object, along with the relative location information provided by the naming convention for each label, 
# may be used to determine the pixel coordinates for each object in the full, corresponding native resolution tile. 
# The pixel coordinates may be translated to geocoordinates using the EPSG:32633 coordinate system and the following geotransform for each tile:

# Tile 1: (307670.04, 0.31, 0.0, 5434427.100000001, 0.0, -0.31)
# Tile 2: (312749.07999999996, 0.31, 0.0, 5403952.860000001, 0.0, -0.31)
# Tile 3: (312749.07999999996, 0.31, 0.0, 5363320.540000001, 0.0, -0.31)
# see here on gdal format geotransform: https://gdal.org/en/stable/tutorials/geotransforms_tut.html

# look into adding dataset crs or projection to metadata dict
dataset_metadata = {
    'deu_maxar_vhr_2023': {
        'doi': '10.6084/m9.figshare.22081091.v3',
        'repo': 'figshare',
        'compression': 'zip',
        'label_fmt': 'yolo_fmt_txt'
    },
    'uk_crowdsourced_pv_2020': {
        'doi': '10.5281/zenodo.4059881',
        'repo': 'zenodo',
        'compression': None,
        'label_fmt': 'geojson'
    },
    # note for report later: Maxar Technologies (MT) was primarily used to determine the extent of solar arrays
    'usa_eia_large_scale_pv_2023': {
        'doi': '10.5281/zenodo.8038684',
        'repo': 'sciencebase',
        'compression': 'zip',
        'label_fmt': 'shapefile'
    },
    'chn_med_res_pv_2024': {
        # using github files since zenodo shapefiles fail to load in QGIS
        'doi': 'https://github.com/qingfengxitu/ChinaPV/tree/main',
        'repo': 'github',
        'compression': None,
        'label_fmt': 'shapefile'
    },
    'usa_cali_usgs_pv_2016': {
        'doi': '10.6084/m9.figshare.3385780.v4',
        'repo': 'figshare',
        'compression': None,
        'label_fmt': 'geojson'
    },
    'chn_jiangsu_vhr_pv_2021': {
        'doi': '10.5281/zenodo.5171712',
        'repo': 'zenodo',
        'compression': 'zip',
        # look into geotransform details for processing these labels
        'label_fmt': 'pixel_mask'
    },
    'ind_pv_solar_farms_2022': {
        'doi': 'https://raw.githubusercontent.com/microsoft/solar-farms-mapping/refs/heads/main/data/solar_farms_india_2021_merged_simplified.geojson',
        'repo': 'github',
        'compression': None,
        'label_fmt': 'geojson'
    },
    'fra_west_eur_pv_installations_2023': {
        'doi': '10.5281/zenodo.6865878',
        'repo': 'zenodo',
        'compression': 'zip',
        'label_fmt': 'geojson'
    },
    'global_pv_inventory_sent2_spot_2021': {
        'doi': '10.5281/zenodo.5005867',
        'repo': 'zenodo',
        'compression': None,
        'label_fmt': 'geojson'
    },
    'global_pv_inventory_sent2_2024': {
        'doi': '10.5281/zenodo.10939099',
        'repo': 'zenodo',
        'compression': None,
        'label_fmt': 'tf_record'
    }

}

default_checked = [
    # 'global_pv_inventory_sent2_2024',
    'global_pv_inventory_sent2_spot_2021',
    'fra_west_eur_pv_installations_2023',
    'ind_pv_solar_farms_2022',
    'usa_cali_usgs_pv_2016',
    'chn_med_res_pv_2024',
    'usa_eia_large_scale_pv_2023',
    'uk_crowdsourced_pv_2020',
    'deu_maxar_vhr_2023'   
]

# display interactive SelectMultiple widget 
data_select = widgets.SelectMultiple(
    options=list(dataset_metadata.keys()),
    value=default_checked,
    description='Select Datasets to fetch and preprocess into georeferenced vectors',
    disabled=False
)
# display the widget
display(data_select)